In [1]:
import numpy as np
from scipy.spatial.distance import pdist, cdist, squareform
from numba import njit,jit

In [2]:
import pickle

In [3]:
var = pickle.load(open(r'C:\Users\mathe\OneDrive\Documentos\GitHub\Anomaly_Detection_sem_CUDA\variables.pickle', 'rb'))

ModelNumber = var['ModelNumber']
Center_samples = var['Center_samples']
Uniquesample = var['Uniquesample']
grid_trad = var['grid_trad']
grid_angl = var['grid_angl']
distancetype = var['distancetype']

In [4]:
print("----- Model Number -----")
print(ModelNumber)

print("----- Center Samples -----")
print(Center_samples)

print("----- Unique Sample -----")
print(Uniquesample)

print("----- Grid Trad -----")
print(grid_trad)

print("----- Grid Angl -----")
print(grid_angl)

print("----- Distance Type -----")
print(distancetype)

----- Model Number -----
2
----- Center Samples -----
[array([14.95026657, -2.49513858,  1.08329909, -5.82066218,  5.61467657,
       -0.77966473,  1.33400379,  0.14598842]), array([-1.73040047,  0.40371659,  1.64941708, -0.28082506, -0.22367741,
        0.12583363, -0.07048597,  0.0463225 ])]
----- Unique Sample -----
[[-1.84026559 -0.12591244  2.13015384 ... -0.77824847  0.82164727
  -0.62003247]
 [-2.93319786 -0.49654047 -1.85497237 ...  1.41004012  0.34678584
  -1.52839328]
 [ 1.70276793 -2.09179944  2.89820802 ...  0.88465257  1.0151653
  -1.40561511]
 ...
 [-2.40967718  0.24114512 -0.83296564 ...  0.05403541 -0.46025521
   0.73014219]
 [-0.78822922  2.06825795  1.99121142 ... -0.42581088 -0.37899894
   1.35986318]
 [-0.75721494 -0.78088933 -0.75608881 ...  1.11151923 -0.67946806
  -0.46372378]]
----- Grid Trad -----
[[6.93684906]]
----- Grid Angl -----
[[0.99039805]]
----- Distance Type -----
euclidean


In [5]:
def cloud_member_recruitment_std(ModelNumber,Center_samples,Uniquesample,grid_trad,grid_angl, distancetype):
    L, W = Uniquesample.shape
    Membership = np.zeros((L,ModelNumber))
    Members = np.zeros((L,ModelNumber*W))
    Count = []

    distance1 = cdist(Uniquesample,Center_samples, metric=distancetype)/grid_trad
    
    distance2 = np.sqrt(cdist(Uniquesample, Center_samples, metric='cosine'))/grid_angl

    distance3 = distance1 + distance2
    print(distance3.shape)
    B = distance3.argmin(1)
    
    
    for i in range(ModelNumber):
        seq = []
        for j,b in enumerate(B):
            if b == i:
                seq.append(j)
        Count.append(len(seq))
        Membership[:Count[i]:,i] = seq
        
        
        for ii, j in zip(range(Count[i]), seq):
            for k in range(W):
                Members[ii,W*i+k] = Uniquesample[j,k]
        
        
    MemberNumber = Count

    return Members,MemberNumber,Membership,B 

In [6]:
%%time

Members_std, MemberNumber_std, Membership_std, B_std  = cloud_member_recruitment_std(ModelNumber,Center_samples,Uniquesample,grid_trad,grid_angl, distancetype)

(299, 2)
Wall time: 5.98 ms


In [7]:
print("----- Members -----")
print(Members_std)

print("----- Member Number -----")
print(MemberNumber_std)

print("----- Membership -----")
print(Membership_std)

print("----- B -----")
print(B_std)

----- Members -----
[[ 7.64915362  6.01679212  2.28661616 ... -0.77824847  0.82164727
  -0.62003247]
 [ 8.1040549  -4.73658298  1.94978232 ...  1.41004012  0.34678584
  -1.52839328]
 [ 9.29097232  6.39343343  2.50659111 ...  0.88465257  1.0151653
  -1.40561511]
 ...
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]]
----- Member Number -----
[11, 288]
----- Membership -----
[[ 24.   0.]
 [ 56.   1.]
 [ 64.   2.]
 [ 65.   3.]
 [ 82.   4.]
 [109.   5.]
 [130.   6.]
 [131.   7.]
 [181.   8.]
 [194.   9.]
 [233.  10.]
 [  0.  11.]
 [  0.  12.]
 [  0.  13.]
 [  0.  14.]
 [  0.  15.]
 [  0.  16.]
 [  0.  17.]
 [  0.  18.]
 [  0.  19.]
 [  0.  20.]
 [  0.  21.]
 [  0.  22.]
 [  0.  23.]
 [  0.  25.]
 [  0.  26.]
 [  0.  27.]
 [  0.  28.]
 [  0.  29.]
 [  0.  30.]
 [  0.  31.]
 [  0.  32.]
 [  0.  33.]
 [  0.  34.]
 [  0.  35.]

In [30]:
@njit
def cloud_member_recruitment_njit(ModelNumber,Center_samples,Uniquesample,grid_trad,grid_angl, distancetype):
    L, W = Uniquesample.shape
    Membership = np.zeros((L,ModelNumber))
    Members = np.zeros((L,ModelNumber*W))
    Count = []
    
    distance1 = []
    for ii in range(L):
        aux2 = []
        for j in range (len(Center_samples)):
            aux1 = []
            bux1 = 0 
            for k in range(W):
                aux1.append((Center_samples[j][k]-Uniquesample[ii,k])**2)
                bux1 += ((Center_samples[j][k]-Uniquesample[ii,k])**2)
            aux2.append((bux1**(0.5))/grid_trad)
        distance1.append(aux2)

    
    distance2 = []
    for ii in range(L):
        aux1 = []
        for j in range(len(Center_samples)):
            dot = 0
            denom_a = 0
            denom_b = 0
            for k in range (len(Center_samples[j])):
                dot += (Center_samples[j][k]*Uniquesample[ii,k])
                denom_a += (Center_samples[j][k] * Center_samples[j][k])
                denom_b += (Uniquesample[ii,k] * Uniquesample[ii,k])

            aux1.append(((1 - ((dot / ((denom_a ** 0.5) * (denom_b ** 0.5)))))**0.5)/grid_angl)
        distance2.append(aux1)
    
    distance3 = []
    for i in range(len(distance1)):
        aux = []
        for j in range(len(distance1[0])):
            aux.append(distance1[i][j] + distance2[i][j])
        distance3.append(aux)
    
    
    B = []
    for dist3 in distance3:
        mini = dist3[0]
        mini_idx = 0
        for ii in range(1, len(dist3)):
            if dist3[ii] < mini:
                mini = dist3[ii]
                mini_idx = ii
        B.append(mini_idx)
    
    for i in range(ModelNumber):
        seq = []
        for j,b in enumerate(B):
            if b == i:
                seq.append(j)
        Count.append(len(seq))
        for ii, j in zip(range(min(Count[i],L)), seq):
            Membership[ii,i] = j
            for k in range(W):
                Members[ii,W*i+k] = Uniquesample[j,k]
    
    MemberNumber = Count
    ret_B = np.array(B).reshape(-1,1)
    return Members,MemberNumber,Membership,ret_B 

In [33]:
%%time

Members_njit, MemberNumber_njit, Membership_njit, B_njit  = cloud_member_recruitment_njit(ModelNumber,Center_samples,Uniquesample,grid_trad,grid_angl, distancetype)

Wall time: 1.99 ms


In [32]:
print("----- Members -----")
print(Members_njit)

print("----- Member Number -----")
print(MemberNumber_njit)

print("----- Membership -----")
print(Membership_njit)

print("----- B -----")
print(B_njit)

----- Members -----
[[ 7.64915362  6.01679212  2.28661616 ... -0.77824847  0.82164727
  -0.62003247]
 [ 8.1040549  -4.73658298  1.94978232 ...  1.41004012  0.34678584
  -1.52839328]
 [ 9.29097232  6.39343343  2.50659111 ...  0.88465257  1.0151653
  -1.40561511]
 ...
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]]
----- Member Number -----
[11, 288]
----- Membership -----
[[ 24.   0.]
 [ 56.   1.]
 [ 64.   2.]
 [ 65.   3.]
 [ 82.   4.]
 [109.   5.]
 [130.   6.]
 [131.   7.]
 [181.   8.]
 [194.   9.]
 [233.  10.]
 [  0.  11.]
 [  0.  12.]
 [  0.  13.]
 [  0.  14.]
 [  0.  15.]
 [  0.  16.]
 [  0.  17.]
 [  0.  18.]
 [  0.  19.]
 [  0.  20.]
 [  0.  21.]
 [  0.  22.]
 [  0.  23.]
 [  0.  25.]
 [  0.  26.]
 [  0.  27.]
 [  0.  28.]
 [  0.  29.]
 [  0.  30.]
 [  0.  31.]
 [  0.  32.]
 [  0.  33.]
 [  0.  34.]
 [  0.  35.]

In [29]:
print("----- Members -----")
print(sum(Members_std - Members_njit))
print("STD Type: ", type(Members_std))
print("NJIT Type: ", type(Members_njit))

print("----- Member Number -----")
print([x1 - x2 for (x1, x2) in zip(MemberNumber_std, MemberNumber_njit)])
print("STD Type: ", type(MemberNumber_std))
print("NJIT Type: ", type(MemberNumber_njit))

print("----- Membership -----")
print(sum(Membership_std - Membership_njit))
print("STD Type: ", type(Membership_std))
print("NJIT Type: ", type(Membership_njit))

B_njit = np.array(B_njit).reshape(-1,1)
print("----- B -----")
print(sum(B_std - B_njit))
print("STD Type: ", type(B_std))
print("NJIT Type: ", type(B_njit))

----- Members -----
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
STD Type:  <class 'numpy.ndarray'>
NJIT Type:  <class 'numpy.ndarray'>
----- Member Number -----
[0, 0]
STD Type:  <class 'list'>
NJIT Type:  <class 'list'>
----- Membership -----
[0. 0.]
STD Type:  <class 'numpy.ndarray'>
NJIT Type:  <class 'numpy.ndarray'>
----- B -----
[[0]]
STD Type:  <class 'numpy.matrix'>
NJIT Type:  <class 'numpy.ndarray'>


#### ATENTAR PARA ESSA MUDANÇA DE np.matrix PARA np.array VER O IMPACTO EM OUTRAS FUNÇÕES DO SODA